In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
 # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, cv, Pool
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e11/sample_submission.csv
/kaggle/input/playground-series-s4e11/train.csv
/kaggle/input/playground-series-s4e11/test.csv


# steps to implement : 
1. importing libraries
2. reading file and exploratory data analysis
3. data wrangling
4. plt and sns
5. correlation heatmap
6. feature engineering and data preprocessing
7. ML model implementation
8. accuracy
9. 

# reading file and exploratory data analysis 


In [2]:
df = pd.read_csv('/kaggle/input/playground-series-s4e11/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e11/test.csv') 
sub = pd.read_csv('/kaggle/input/playground-series-s4e11/sample_submission.csv')
df.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,0,Aaradhya,Female,49.0,Ludhiana,Working Professional,Chef,NaN,5.0,NaN,NaN,2.0,More than 8 hours,Healthy,BHM,No,1.0,2.0,No,0
1,1,Vivan,Male,26.0,Varanasi,Working Professional,Teacher,NaN,4.0,NaN,NaN,3.0,Less than 5 hours,Unhealthy,LLB,Yes,7.0,3.0,No,1
2,2,Yuvraj,Male,33.0,Visakhapatnam,Student,NaN,5.0,NaN,8.97,2.0,NaN,5-6 hours,Healthy,B.Pharm,Yes,3.0,1.0,No,1
3,3,Yuvraj,Male,22.0,Mumbai,Working Professional,Teacher,NaN,5.0,NaN,NaN,1.0,Less than 5 hours,Moderate,BBA,Yes,10.0,1.0,Yes,1
4,4,Rhea,Female,30.0,Kanpur,Working Professional,Business Analyst,NaN,1.0,NaN,NaN,1.0,5-6 hours,Unhealthy,BBA,Yes,9.0,4.0,Yes,0


In [3]:
df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140700 entries, 0 to 140699
Data columns (total 20 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     140700 non-null  int64  
 1   Name                                   140700 non-null  object 
 2   Gender                                 140700 non-null  object 
 3   Age                                    140700 non-null  float64
 4   City                                   140700 non-null  object 
 5   Working Professional or Student        140700 non-null  object 
 6   Profession                             104070 non-null  object 
 7   Academic Pressure                      27897 non-null   float64
 8   Work Pressure                          112782 non-null  float64
 9   CGPA                                   27898 non-null   float64
 10  Study Satisfaction                     27897 non-null   

In [4]:
#splitting datasets into testing and validation


In [5]:
df.describe()

,id,Age,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Work/Study Hours,Financial Stress,Depression
count,140700.000000,140700.000000,27897.000000,112782.000000,27898.000000,27897.000000,112790.000000,140700.000000,140696.000000,140700.000000
mean,70349.500000,40.388621,3.142273,2.998998,7.658636,2.944940,2.974404,6.252679,2.988983,0.181713
std,40616.735775,12.384099,1.380457,1.405771,1.464466,1.360197,1.416078,3.853615,1.413633,0.385609
min,0.000000,18.000000,1.000000,1.000000,5.030000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,35174.750000,29.000000,2.000000,2.000000,6.290000,2.000000,2.000000,3.000000,2.000000,0.000000
50%,70349.500000,42.000000,3.000000,3.000000,7.770000,3.000000,3.000000,6.000000,3.000000,0.000000
75%,105524.250000,51.000000,4.000000,4.000000,8.920000,4.000000,4.000000,10.000000,4.000000,0.000000
max,140699.000000,60.000000,5.000000,5.000000,10.000000,5.000000,5.000000,12.000000,5.000000,1.000000


# data wrangling 


In [6]:
#checking how many missing values 
df.isnull().sum()

id                                            0
Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                36630
Academic Pressure                        112803
Work Pressure                             27918
CGPA                                     112802
Study Satisfaction                       112803
Job Satisfaction                          27910
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64

In [7]:
df.columns


Index(['id', 'Name', 'Gender', 'Age', 'City',
       'Working Professional or Student', 'Profession', 'Academic Pressure',
       'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction',
       'Sleep Duration', 'Dietary Habits', 'Degree',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours',
       'Financial Stress', 'Family History of Mental Illness', 'Depression'],
      dtype='object')

In [8]:
df.dtypes

id                                         int64
Name                                      object
Gender                                    object
Age                                      float64
City                                      object
Working Professional or Student           object
Profession                                object
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                            object
Dietary Habits                            object
Degree                                    object
Have you ever had suicidal thoughts ?     object
Work/Study Hours                         float64
Financial Stress                         float64
Family History of Mental Illness          object
Depression                                 int64
dtype: object

In [9]:
df = df.copy()

# Drop irrelevant columns
# Uncomment below if you need to drop these columns
# df = df.drop(columns=['id', 'Name'])

# Impute missing values for numerical columns using .loc
df.loc[:, 'Academic Pressure'] = df['Academic Pressure'].fillna(df['Academic Pressure'].median())
df.loc[:, 'Work Pressure'] = df['Work Pressure'].fillna(df['Work Pressure'].median())
df.loc[:, 'CGPA'] = df['CGPA'].fillna(df['CGPA'].median())
df.loc[:, 'Study Satisfaction'] = df['Study Satisfaction'].fillna(df['Study Satisfaction'].median())
df.loc[:, 'Job Satisfaction'] = df['Job Satisfaction'].fillna(df['Job Satisfaction'].median())

# Impute missing values for categorical columns
df.loc[:, 'Profession'] = df['Profession'].fillna('unknown')

# Drop rows where specific columns have null values
df = df.dropna(subset=['Dietary Habits', 'Degree', 'Financial Stress'])

df.isnull().sum()

# note : df[column] = .... doesn't work because it modifies a view not the actual column itself 

id                                       0
Name                                     0
Gender                                   0
Age                                      0
City                                     0
Working Professional or Student          0
Profession                               0
Academic Pressure                        0
Work Pressure                            0
CGPA                                     0
Study Satisfaction                       0
Job Satisfaction                         0
Sleep Duration                           0
Dietary Habits                           0
Degree                                   0
Have you ever had suicidal thoughts ?    0
Work/Study Hours                         0
Financial Stress                         0
Family History of Mental Illness         0
Depression                               0
dtype: int64

In [10]:
# Initialize the OneHotEncoder
#encoder = OneHotEncoder(drop='first', sparse=False)

# Fit and transform the categorical columns
#encoded_columns = encoder.fit_transform(df[['City', 'Profession','Working Professional or Student', 'Degree', 'Gender', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']])

# Create a DataFrame with the encoded columns
#encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['City', 'Profession','Working Professional or Student', 'Degree', 'Gender', 'Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']))

# Combine the original DataFrame (excluding the categorical columns) with the encoded DataFrame
#df_encoded = pd.concat([df.drop(columns=['City', 'Profession','Working Professional or Student', 'Degree', 'Gender','Have you ever had suicidal thoughts ?', 'Family History of Mental Illness']), encoded_df], axis=1)

#print("\nDataFrame after One-Hot Encoding:")
#df_encoded.head()

# this method then creates a LOT of columns (high cardinality) so instead of one hot encoding we should use catboost or LGBM as these models are goot for categorical data with high cardinality 


## catboost / lgbm 

we are going to use catboost / lgbm instead of one hot encoding because one hot encoding creates a lot of columns (high cardinality) 

In [11]:
#columns (categorical) family history of mental illness,
#have you ever had suicidal thoughts, degree ,
#dietry habits, sleep duration,profession ,
#working professional or student, city , gender, name 

columns_to_convert = [
    'Name',
    'Gender',
    'City',
    'Working Professional or Student',
    'Profession',
    'Sleep Duration',
    'Dietary Habits',
    'Degree',
    'Have you ever had suicidal thoughts ?',
    'Family History of Mental Illness', 
    
]

df.loc[:, columns_to_convert] = df.loc[:, columns_to_convert].astype('string')
df.dtypes

id                                         int64
Name                                      object
Gender                                    object
Age                                      float64
City                                      object
Working Professional or Student           object
Profession                                object
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                            object
Dietary Habits                            object
Degree                                    object
Have you ever had suicidal thoughts ?     object
Work/Study Hours                         float64
Financial Stress                         float64
Family History of Mental Illness          object
Depression                                 int64
dtype: object

In [12]:
X = df.drop(columns=['Name', 'Depression'])
y = df['Depression']
cat_features = [
    'Gender',
    'City',
    'Working Professional or Student',
    'Profession',
    'Sleep Duration',
    'Dietary Habits',
    'Degree',
    'Have you ever had suicidal thoughts ?',
    'Family History of Mental Illness', 
    
]
# Initialize CatBoostClassifier
model = CatBoostClassifier(cat_features=cat_features, iterations=201, learning_rate=0.1, depth=3, verbose=0, early_stopping_rounds=50)

# Fit the model
model.fit(X, y)

# Make predictions
predictions = model.predict(X)
print("Predictions:", predictions)



Predictions: [0 1 1 ... 0 0 1]


In [13]:


accuracy = accuracy_score(y, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.9393133840358234


In [14]:
# Convert the specified columns to 'category' dtype : this is because lgbm need categorical variables under category data type 
df[columns_to_convert] = df[columns_to_convert].astype('category')

# Verify the data types to make sure the conversion worked
print(df.dtypes)

X = df.drop(columns=['Name', 'Depression'])
y = df['Depression']
# Train-Test Split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

lgbm_model = LGBMClassifier(
    n_estimators=500, 
    learning_rate=0.1, 
    max_depth=6, 
    num_leaves=64,  # Increased number of leaves for better model flexibility
    class_weight='balanced',  # Handle class imbalance
    random_state=42
)

#fitting the LGBM model 
lgbm_model.fit(X_train, y_train)

#making prediction with the lightGBM model 
y_pred_lgbm = lgbm_model.predict(X_test)
y_proba_lgbm = lgbm_model.predict_proba(X_test)[:,1]




id                                          int64
Name                                     category
Gender                                   category
Age                                       float64
City                                     category
Working Professional or Student          category
Profession                               category
Academic Pressure                         float64
Work Pressure                             float64
CGPA                                      float64
Study Satisfaction                        float64
Job Satisfaction                          float64
Sleep Duration                           category
Dietary Habits                           category
Degree                                   category
Have you ever had suicidal thoughts ?    category
Work/Study Hours                          float64
Financial Stress                          float64
Family History of Mental Illness         category
Depression                                  int64


In [15]:
accuracy = accuracy_score(y_test, y_pred_lgbm)
print("Accuracy:", accuracy)


Accuracy: 0.9312673253251831


## metal model and combining stuff 

In [16]:
meta_model = LogisticRegression()

# Stacking classifier combining the base models and meta-model
stacking_model = StackingClassifier(estimators=[('lgbm', lgbm_model), ('catboost', model)], final_estimator=meta_model)

# Fit the stacking model
stacking_model.fit(X_train, y_train)

# Predict with the stacking model
y_pred = stacking_model.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 20508, number of negative: 92044
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011437 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 664
[LightGBM] [Info] Number of data points in the train set: 112552, number of used features: 18
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [17]:
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')

Accuracy: 0.9410405856848391


## Cross validation 

In [18]:
data_pool = Pool(X, label=y, cat_features=cat_features)

# Set the parameters for the CatBoost model
params = {
    'iterations': 1000,           # Number of boosting iterations
    'learning_rate': 0.05,        # Learning rate
    'depth': 8,                   # Depth of trees
    'loss_function': 'Logloss',   # Logloss for binary classification
    'eval_metric': 'Accuracy',    # Accuracy for evaluation (added inside params)
    'cat_features': cat_features,  # Specify which columns are categorical
    'random_seed': 42            # To ensure reproducibility
}

# Perform cross-validation (5-fold in this case)
cv_results = cv(
    data_pool,            # Pool object with data and target
    params,               # Parameters for the CatBoost model
    fold_count=5,         # Number of folds (e.g., 5-fold cross-validation)
    shuffle=True,         # Shuffle the data before splitting into folds
    partition_random_seed=42,  # Random seed for partitioning the data
    early_stopping_rounds=50  # Stop if there's no improvement after 50 rounds
)

# Display the results of cross-validation


Training on fold [0/5]
0:	learn: 0.9239989	test: 0.9235581	best: 0.9235581 (0)	total: 390ms	remaining: 6m 29s
1:	learn: 0.9262823	test: 0.9262234	best: 0.9262234 (1)	total: 805ms	remaining: 6m 41s
2:	learn: 0.9288767	test: 0.9284267	best: 0.9284267 (2)	total: 1.2s	remaining: 6m 39s
3:	learn: 0.9283702	test: 0.9288887	best: 0.9288887 (3)	total: 1.59s	remaining: 6m 35s
4:	learn: 0.9285657	test: 0.9289598	best: 0.9289598 (4)	total: 1.98s	remaining: 6m 33s
5:	learn: 0.9284236	test: 0.9291730	best: 0.9291730 (5)	total: 2.38s	remaining: 6m 33s
6:	learn: 0.9284147	test: 0.9287466	best: 0.9291730 (5)	total: 2.74s	remaining: 6m 29s
7:	learn: 0.9295431	test: 0.9292086	best: 0.9292086 (7)	total: 3.13s	remaining: 6m 28s
8:	learn: 0.9304493	test: 0.9299549	best: 0.9299549 (8)	total: 3.5s	remaining: 6m 25s
9:	learn: 0.9308669	test: 0.9303813	best: 0.9303813 (9)	total: 3.84s	remaining: 6m 20s
10:	learn: 0.9312489	test: 0.9313053	best: 0.9313053 (10)	total: 4.22s	remaining: 6m 19s
11:	learn: 0.9312845

## Submission 


In [19]:
submission = pd.DataFrame({
    'id': test.iloc[:len(y_pred)]['id'],  # Slice the 'test' DataFrame to match 'y_pred'
    'Depression': y_pred
})
submission.to_csv('submission5.csv', index=False)

submission.head()

,id,Depression
0,140700,0
1,140701,0
2,140702,1
3,140703,0
4,140704,0
